In [2]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

vec = cPickle.load(open('../../preprocess/allfields_with_embedding_19995.p', 'rb'))
cohenvec = cPickle.load(open('../../preprocess/cohendata_dedup_19995.p', 'rb'))

In [3]:
import numpy as np
index = vec.index['abstract']
vec.X = vec.X[index[0]:index[1]]
X_tf = np.zeros((vec.X.shape[0], vec.vocab_size))
for i in range(len(vec.X)) :
    X_tf[i, vec.X[i, :]] = 1.

X_tf = X_tf[:, 2:]
X_tf.shape

(41784, 19993)

In [4]:
X = cohenvec.X
cohen_X_tf = np.zeros((X.shape[0], vec.vocab_size))
for i in range(len(X)) :
    cohen_X_tf[i, X[i, :]] = 1.

cohen_X_tf = cohen_X_tf[:, 2:]
cohen_X_tf.shape

(1767, 19993)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X_tf1 = tfidf.fit_transform(X_tf)

In [6]:
cohen_tf = tfidf.transform(cohen_X_tf)
cohen_tf = TfidfTransformer().fit_transform(cohen_X_tf)

In [7]:
cohen_tf

<1767x19993 sparse matrix of type '<type 'numpy.float64'>'
	with 210561 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
scores = cosine_similarity(X=cohen_tf)

In [10]:
import pandas as  pd
df = pd.read_csv('../data/files/test_cohen_dedup.csv')

nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1

H[np.arange(nb_studies), np.arange(nb_studies)] = 0
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [30]:
from sklearn.metrics import roc_auc_score, average_precision_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print rocs

0.767865083944
{'NSAIDS_processed': 0.84607727856817461, 'ACEInhibitors_processed': 0.78527562953992291, 'SkeletalMuscleRelaxants_processed': 0.61983749889605222, 'Triptans_processed': 0.88122951639153591, 'OralHypoglycemics_processed': 0.84002497921620933, 'CalciumChannelBlockers_processed': 0.63575819980555226, 'BetaBlockers_processed': 0.65591328470712751, 'Estrogens_processed': 0.86704321319705935, 'ADHD_processed': 0.8480638147623073, 'Statins_processed': 0.7637934123253175, 'Antihistamines_processed': 0.68688915375446957, 'ProtonPumpInhibitors_processed': 0.7763995404500238, 'Opiods_processed': 0.75264380640332174, 'AtypicalAntipsychotics_processed': 0.75785991943538467}


In [10]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print rocs

0.804260448065
{'NSAIDS_processed': 0.86517318779449548, 'ACEInhibitors_processed': 0.81968083913475587, 'SkeletalMuscleRelaxants_processed': 0.62530424799081519, 'Triptans_processed': 0.94105541433096329, 'OralHypoglycemics_processed': 0.87411345049591116, 'CalciumChannelBlockers_processed': 0.65725097841329017, 'BetaBlockers_processed': 0.67602194330417575, 'Estrogens_processed': 0.87664643049258439, 'ADHD_processed': 0.87700506107473108, 'Statins_processed': 0.79644987024324665, 'Antihistamines_processed': 0.71046702646721394, 'ProtonPumpInhibitors_processed': 0.84116156951087551, 'Opiods_processed': 0.75516191901930607, 'AtypicalAntipsychotics_processed': 0.81041986112189435}


In [11]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print rocs

0.788580988083
{'NSAIDS_processed': 0.86081039785636881, 'ACEInhibitors_processed': 0.81779121606787486, 'SkeletalMuscleRelaxants_processed': 0.62686832111631197, 'Triptans_processed': 0.88204816503315897, 'OralHypoglycemics_processed': 0.84860039054192526, 'CalciumChannelBlockers_processed': 0.65248309884956812, 'BetaBlockers_processed': 0.6828057486333452, 'Estrogens_processed': 0.88136744911892839, 'ADHD_processed': 0.90912140042713196, 'Statins_processed': 0.80877777014674845, 'Antihistamines_processed': 0.75519940063002311, 'ProtonPumpInhibitors_processed': 0.80235795794014053, 'Opiods_processed': 0.75069060968548307, 'AtypicalAntipsychotics_processed': 0.74073539362639651}
